In [1]:
from fastcore.all import *
from fastprogress import progress_bar
import random, json

random.seed(0)

In [2]:
%cd /storage/Data/Yelp

/storage/Data/Yelp


In [3]:
photos = []
with open('photos.json') as f:
    for line in f:
        photos.append(json.loads(line))
photos = L(photos)

In [4]:
photos[0]

{'photo_id': '001kKc6P_rmgaXThAHxEkw',
 'business_id': 'zKZc8GwCaKbACaV0HZAmpg',
 'caption': 'Clean and simple interior',
 'label': 'inside'}

In [5]:
id2entry = {photo['photo_id']: photo for photo in photos}

In [6]:
all_photos = Path('photos').ls()

In [7]:
all_photos.shuffle();

In [8]:
all_photos[0]

Path('photos/_aow0hgEbtWME4VsSqpjgA.jpg')

In [9]:
output_dir = Path('subsets')
output_dir.mkdir(exist_ok=True)
for i, batch in enumerate(progress_bar(list(chunked(all_photos, chunk_sz=5_000, drop_last=False)))):
    # Write a subset photos.json
    json_data = [id2entry[o.name[:-4]] for o in batch]
    json_file = (output_dir / f'subset{i:03d}-photos.json')
    json_file.write_text('\n'.join(json.dumps(p) for p in json_data))

    # Construct a list of filenames to put in the tar file
    filenames = '\0'.join(L(batch).map(str) + str(json_file))
    source_file = output_dir / 'files.txt'
    Path(source_file).write_text(filenames, encoding = 'utf-8')
    run(['tar', 'zcf', output_dir / f'yelp_photos_subset{i:03d}.tgz', '--null', '--files-from', source_file, '--xform', f's:subsets/subset...-::;s:^:yelp_photos_subset{i:03d}/:'], stderr=True)
    source_file.unlink()
    json_file.unlink()
